# League of Legends Analysis
# Vy Nguyen and Daniel Strub
# CPSC 322

In [14]:
import importlib
import sys
import os
import copy
import random
from tabulate import tabulate

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [15]:
game_fname = os.path.join("input_data", "games_small.csv")
game_table = MyPyTable()
game_table.load_from_file(game_fname)

winner = game_table.get_column("winner")
first_Blood = game_table.get_column("firstBlood")
first_Tower = game_table.get_column("firstTower")
first_Inhibitor = game_table.get_column("firstInhibitor")
first_Baron = game_table.get_column("firstBaron")
first_Dragon = game_table.get_column("firstDragon")
first_RiftHerald = game_table.get_column("firstRiftHerald")

game_X = [[first_Blood[x], first_Tower[x], first_Inhibitor[x], first_Baron[x], first_Dragon[x], first_RiftHerald[x]] for x in range(len(winner))]
game_Y = [y for y in winner]


In [16]:
print(str(round(myutils.get_win_count(game_table, "winner", "firstBlood") * 100, 2)) + "% of games with first blood are won")

print(str(round(myutils.get_win_count(game_table, "winner", "firstTower") * 100, 2)) + "% of games with first tower are won")

print(str(round(myutils.get_win_count(game_table, "winner", "firstInhibitor") * 100, 2)) + "% of games with first inhibitor are won")

print(str(round(myutils.get_win_count(game_table, "winner", "firstBaron") * 100, 2)) + "% of games with first Baron are won")

print(str(round(myutils.get_win_count(game_table, "winner", "firstDragon") * 100, 2)) + "% of games with first dragon are won")

print(str(round(myutils.get_win_count(game_table, "winner", "firstRiftHerald") * 100, 2)) + "% of games with first Rift Herald are won")

58.56% of games with first blood are won
68.92% of games with first tower are won
79.83% of games with first inhibitor are won
49.1% of games with first Baron are won
65.6% of games with first dragon are won
34.0% of games with first Rift Herald are won


In [17]:
# KNNeightbors
test_size = 2000
X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(game_X), copy.deepcopy(game_Y), test_size=test_size, shuffle=True)

nb = MyKNeighborsClassifier()
nb.fit(X_train, y_train)
predictions = []
for i, x in enumerate(X_test):
    prediction = nb.predict([x])
    predictions.append(prediction[0])

acc = round(sum([int(x==y) for x,y in zip(predictions, y_test)])/len(predictions), 2)
print("KNN:" )
print("Accuracy = " + str(acc))
print("Error rate = " + str(1-acc))

KNN:
Accuracy = 0.53
Error rate = 0.47


In [18]:
test_size = 2000

X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(game_X), copy.deepcopy(game_Y), test_size=test_size, shuffle=True)

nb = MyNaiveBayesClassifier()
nb.fit(X_train, y_train)
predictions = []
for i, x in enumerate(X_test):
    prediction = nb.predict([x])
    predictions.append(prediction[0])
# Result
acc = round(sum([int(x==y) for x,y in zip(predictions, y_test)])/len(predictions), 2)
print("Naive Bayes: ")
print("Accuracy = " + str(acc))
print("Error rate = " + str(1-acc))

Naive Bayes: 
Accuracy = 0.88
Error rate = 0.12


In [19]:
X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(game_X), copy.deepcopy(game_Y), test_size=2000)

best_trees = []
max_accuracy = .50

for i in range(20):
    F = random.randint(1,5)
    N = random.randint(1,20)
    M = random.randint(1,N)

    rf = MyRandomForestClassifier()
    rf.fit(game_X, game_Y, F=1, N=5, M=4)

    predictions = []
    for i, x in enumerate(X_test):
        prediction = rf.predict([x])
        # print(prediction, y_test[i])
        # print(prediction)
        predictions.append(int(prediction[0] == y_test[i]))
    if sum(predictions)/len(predictions) > max_accuracy:
        print("F:", F, "N:", N, "M:", M, "Accuracy:", sum(predictions)/len(predictions))
        f = open("best_tree.txt", "w")
        f.write(str(rf.trees))
        f.close()
        max_accuracy = sum(predictions)/len(predictions)
        best_trees = rf.trees

F: 5 N: 4 M: 1 Accuracy: 0.883
F: 3 N: 20 M: 18 Accuracy: 0.895


In [ ]:
import ast

if best_trees == []:
    with open("best_tree.txt", "r") as data:
        best_trees = ast.literal_eval(data.read())

In [ ]:
test_size = 2000

X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(wildfire_X), copy.deepcopy(wildfire_y), test_size=test_size, shuffle=True)

rf = MyRandomForestClassifier()
rf.trees = copy.deepcopy(best_trees)
predictions = []
for i, x in enumerate(X_test):
    sys.stdout.write("\r" + str(i) + "/" + str(len(X_test) -1) + "    ")
    sys.stdout.flush()
    prediction = rf.predict([x])
    predictions.append(prediction[0])

print()
acc = round(sum([int(x==y) for x,y in zip(predictions, y_test)])/len(predictions), 2)
print("Random Forest")
print("Accuracy = ", acc)
print("Error Rate =", str(1-acc))